In [106]:
#######################################################################################################
# Imports

import azure.storage.blob as azureblob
import azure.batch.models as batchmodels
import azure.batch.batch_service_client as batch
import azure.batch.batch_auth as batchauth
import datetime
import os
import sys
import time

#######################################################################################################
# Global keys

_BATCH_ACCOUNT_NAME = "climatebasedbatch"
_BATCH_ACCOUNT_KEY = "W94ukoxG2neFkk6teOVZ3IQ8IQjmPJqPcFq48I9lLzCrPEQSRFS/+euaUEkkSyPoulUgnx5IEZxztA9574Hluw=="
_BATCH_ACCOUNT_URL = "https://climatebasedbatch.westeurope.batch.azure.com"

_STORAGE_ACCOUNT_NAME = "radfiles"
_STORAGE_ACCOUNT_KEY = "aRRVzOkO/kwS35CIwNVIa18aGoMfZD5D3yAy3GlorkkU2G+9q5rAscXoC21IIylJZerBefwMgxYYF3qzquALrw=="

_POOL_ID = "batchpool"
_MIN_POOL_NODE = 1
_MAX_POOL_NODE = 100

_POOL_VM_SIZE = 'BASIC_A1'
_NODE_OS_PUBLISHER = 'Canonical'
_NODE_OS_OFFER = 'UbuntuServer'
_NODE_OS_SKU = '16'

_JOB_NAME = "0000000-testjob-3513"
_JOB_DIRECTORY = "./radfiles"
_COPY_TO_BLOB_LOCAL = "./docker/copy_to_blob.py"

#######################################################################################################
# Create the blob client - for use in managing blob storage containers and upload/download transactions

blob_client = azureblob.BlockBlobService(
    account_name=_STORAGE_ACCOUNT_NAME, 
    account_key=_STORAGE_ACCOUNT_KEY
)
print("Blob client generated:  {0:}\n".format(blob_client))

#######################################################################################################
# Create the batch client - for use in transacting with batch pool and nodes

batch_client = batch.BatchServiceClient(
    batchauth.SharedKeyCredentials(
        _BATCH_ACCOUNT_NAME, 
        _BATCH_ACCOUNT_KEY
    ), 
    base_url=_BATCH_ACCOUNT_URL
)
print("Batch client generated: {0:}\n".format(batch_client))

#######################################################################################################
# Create container for job file storage

# container = blob_client.create_container(
#     _JOB_NAME, 
#     fail_on_exist=False
# )
# print("Container created: [{0:}/{1:}]\n".format(_STORAGE_ACCOUNT_NAME, _JOB_NAME))

#######################################################################################################
# Define the fucntions used to find files and uplaod them to the blob from the host

def directory_files(directory):
    """
    Generates a list of the files within a directory
    :param str directory: A path to a directory.
    :return list: List of files
    """
    return [item for sublist in [[os.path.join(path, name) for name in files] for path, subdirs, files in os.walk(directory)] for item in sublist]

def upload_file_to_container(block_blob_client, container_name, file_path):
    """
    Uploads a local file to an Azure Blob storage container.
    :param block_blob_client: An Azure blockblobservice client.
    :type block_blob_client: `azure.storage.blob.BlockBlobService`
    :param str container_name: The name of the Azure Blob storage container.
    :param str file_path: The local path to the file.
    :rtype: `azure.batch.models.ResourceFile`
    :return: A ResourceFile initialized with a SAS URL appropriate for Batch tasks.
    """
    blob_name = os.path.basename(file_path)
    print('Uploading file {0:} to container [{1:}/{0:}]'.format(blob_name, container_name))
    block_blob_client.create_blob_from_path(
        container_name, 
        blob_name, 
        file_path
    )
    sas_token = block_blob_client.generate_blob_shared_access_signature(
        container_name, 
        blob_name, 
#         permission=azureblob.BlobPermissions.READ, 
        permission=azureblob.BlobPermissions(
            read=True, 
            add=False, 
            create=False, 
            write=False, 
            delete=False, 
            _str=None
        ), 
        expiry=datetime.datetime.utcnow() + datetime.timedelta(hours=24)
    )
    sas_url = block_blob_client.make_blob_url(
        container_name, 
        blob_name, 
        sas_token=sas_token
    )
    return batchmodels.ResourceFile(file_path=blob_name, blob_source=sas_url)

#######################################################################################################
# Upload the files common to all simulations (sky_mtx.json, surfaces.json and copy_to_blob.py)

# _SKY_MTX = upload_file_to_container(
#     blob_client, 
#     _JOB_NAME, 
#     os.path.join(_JOB_DIRECTORY, "sky_mtx.json")
# )
# _SURFACES = upload_file_to_container(
#     blob_client, 
#     _JOB_NAME, 
#     os.path.join(_JOB_DIRECTORY, "surfaces.json")
# )
# _COPY_TO_BLOB = upload_file_to_container(
#     blob_client, 
#     _JOB_NAME, 
#     _COPY_TO_BLOB_LOCAL
# )

#######################################################################################################
# Upload the files unique to each simulation ([analysis_grid_0.json, ..., analysis_grid_n.json])

# _ANALYSIS_GRIDS = []
# _ANALYSIS_GRIDS_NAMES = []
# for _file in directory_files(os.path.join(_JOB_DIRECTORY, "AnalysisGrids")):
#     _ANALYSIS_GRIDS.append(upload_file_to_container(blob_client, _JOB_NAME, _file))
#     _ANALYSIS_GRIDS_NAMES.append(os.path.basename(_file).replace(".json", ""))

# print()

#######################################################################################################
# Generate a sas token and url capable of read/write

container_sas_token = blob_client.generate_container_shared_access_signature(
    _JOB_NAME, 
    permission=azureblob.BlobPermissions(
        read=True, 
        add=True, 
        create=True, 
        write=True, 
        delete=False, 
        _str=None
    ), 
    expiry=datetime.datetime.utcnow() + datetime.timedelta(hours=24)
)

container_sas_url = blob_client.make_container_url(
    _JOB_NAME, 
    sas_token=container_sas_token
)

Blob client generated:  <azure.storage.blob.blockblobservice.BlockBlobService object at 0x000002A3C40F89B0>

Batch client generated: <azure.batch.batch_service_client.BatchServiceClient object at 0x000002A3C41364A8>



AttributeError: module 'azure.storage.blob' has no attribute 'blob_client'

In [6]:
# Get shared access signature providing write access to the container
def get_container_sas_token(block_blob_client, container_name, blob_permissions):
    """
    Obtains a shared access signature granting the specified permissions to the container.
    :param block_blob_client: A blob service client.
    :type block_blob_client: `azure.storage.blob.BlockBlobService`
    :param str container_name: The name of the Azure Blob storage container.
    :param BlobPermissions blob_permissions:
    :rtype: str
    :return: A SAS token granting the specified permissions to the container.
    """
    container_sas_token = block_blob_client.generate_container_shared_access_signature(
        container_name,
        permission=blob_permissions,
        expiry=datetime.datetime.utcnow() + datetime.timedelta(hours=24))
    return container_sas_token

output_container_sas_token = get_container_sas_token(blob_client, _JOB_NAME, azureblob.BlobPermissions.WRITE)
print(output_container_sas_token)

se=2018-08-22T13%3A04%3A00Z&sp=w&sv=2017-04-17&sr=c&sig=h9K8klrifqXQUpM7q54LYTNuf7H8/qPPXSEDRyJe41Y%3D


In [98]:
startup_commands = [
        "curl -fSsL https://bootstrap.pypa.io/get-pip.py | python",  # Install pip
        "pip install azure-storage==0.32.0",  # Install the azure-storage module
        "sudo apt-get install docker -y && sudo apt-get install docker.io -y",  # Install docker
        "sudo docker pull tgerrish/bhrad"  # Pull RadHoneyWhale from docker hub
    ]
"/bin/bash -c \"set -e; set -o pipefail; {}; wait\"".format(";".join(startup_commands))


'/bin/bash -c "set -e; set -o pipefail; curl -fSsL https://bootstrap.pypa.io/get-pip.py | python;pip install azure-storage==0.32.0;sudo apt-get install docker -y && sudo apt-get install docker.io -y;sudo docker pull tgerrish/bhrad; wait"'

In [7]:
# Create the pool containing the compute nodes executing the tasks
def select_latest_verified_vm_image_with_node_agent_sku(batch_client, publisher, offer, sku_starts_with):
    """
    Select the latest verified image that Azure Batch supports given a publisher, offer and sku (starts with filter).
    :param batch_client: The batch client to use.
    :type batch_client: `batchserviceclient.BatchServiceClient`
    :param str publisher: vm image publisher
    :param str offer: vm image offer
    :param str sku_starts_with: vm sku starts with filter
    :rtype: tuple
    :return: (node agent sku id to use, vm image ref to use)
    """
    # get verified vm image list and node agent sku ids from service
    node_agent_skus = batch_client.account.list_node_agent_skus()
    # pick the latest supported sku
    skus_to_use = [
        (sku, image_ref) for sku in node_agent_skus for image_ref in sorted(
            sku.verified_image_references, key=lambda item: item.sku)
        if image_ref.publisher.lower() == publisher.lower() and
        image_ref.offer.lower() == offer.lower() and
        image_ref.sku.startswith(sku_starts_with)
    ]
    # skus are listed in reverse order, pick first for latest
    sku_to_use, image_ref_to_use = skus_to_use[0]
    return (sku_to_use.id, image_ref_to_use)

def wrap_commands_in_shell(ostype, commands):
    """Wrap commands in a shell
    :param list commands: list of commands to wrap
    :param str ostype: OS type, linux or windows
    :rtype: str
    :return: a shell wrapping commands
    """
    if ostype.lower() == "linux":
        return "/bin/bash -c \"set -e; set -o pipefail; {0:}; wait\"".format(";".join(commands))
    elif ostype.lower() == "windows":
        return "cmd.exe /c {0:}".format("&".join(commands))
    else:
        raise ValueError("unknown ostype: {}".format(ostype))
        
def print_batch_exception(batch_exception):
    """
    Prints the contents of the specified Batch exception.
    :param batch_exception:
    """
    print('-------------------------------------------')
    print('Exception encountered:')
    if (batch_exception.error and batch_exception.error.message and
            batch_exception.error.message.value):
        print(batch_exception.error.message.value)
        if batch_exception.error.values:
            print()
            for mesg in batch_exception.error.values:
                print('{}:\t{}'.format(mesg.key, mesg.value))
    print('-------------------------------------------')

def create_pool(batch_service_client, pool_id, resource_files, publisher, offer, sku, node_count):
    """
    Creates a pool of compute nodes with the specified OS settings.
    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str pool_id: An ID for the new pool.
    :param list resource_files: A collection of resource files for the pool's start task.
    :param str publisher: Marketplace image publisher
    :param str offer: Marketplace image offer
    :param str sku: Marketplace image sku
    """
    print('Creating pool [{0:}]...'.format(pool_id))
    # Specify the commands for the pool's start task to be run on each node as it joins the pool.
    commands = [
        "curl -fSsL https://bootstrap.pypa.io/get-pip.py | python",  # Install pip
        "pip install azure-storage==0.32.0",  # Install the azure-storage module
        "sudo apt-get install docker -y && sudo apt-get install docker.io -y",  # Install docker
        "sudo docker pull tgerrish/bhrad"  # Pull RadHoneyWhale from docker hub
    ]
    # Get the node agent SKU and image reference for the virtual machine configuration.
    sku_to_use, image_ref_to_use = select_latest_verified_vm_image_with_node_agent_sku(
        batch_service_client, 
        publisher, 
        offer, 
        sku
    )
    user = batchmodels.AutoUserSpecification(
        scope=batchmodels.AutoUserScope.pool, 
        elevation_level=batchmodels.ElevationLevel.admin
    )
    new_pool = batch.models.PoolAddParameter(
        id=pool_id,
        virtual_machine_configuration=batchmodels.VirtualMachineConfiguration(
            image_reference=image_ref_to_use,
            node_agent_sku_id=sku_to_use),
        vm_size=_POOL_VM_SIZE,
        enable_auto_scale=True,
        auto_scale_formula='pendingTaskSamplePercent =$PendingTasks.GetSamplePercent(180 * TimeInterval_Second);pendingTaskSamples = pendingTaskSamplePercent < 70 ? 1 : avg($PendingTasks.GetSample(180 * TimeInterval_Second)); $TargetDedicatedNodes = min(100, pendingTaskSamples);', 
        auto_scale_evaluation_interval=datetime.timedelta(minutes=5),
        start_task=batch.models.StartTask(
            command_line=wrap_commands_in_shell(
                "linux",
                commands),
            user_identity=batchmodels.UserIdentity(auto_user=user),
            wait_for_success=True,
            resource_files=resource_files),
    )

    try:
        batch_service_client.pool.add(
            new_pool
        )
    except batchmodels.batch_error.BatchErrorException as err:
        print_batch_exception(
            err
        )
        raise

_POOL_NODE_COUNT = len(_ANALYSIS_GRIDS)

pool = create_pool(batch_client, _POOL_ID, [_SKY_MTX, _SURFACES, _COPY_TO_BLOB], _NODE_OS_PUBLISHER, _NODE_OS_OFFER, _NODE_OS_SKU, _POOL_NODE_COUNT)

print("Pool created ...")

Creating pool [testpool]...
Pool created ...


In [9]:
if batch_client.pool.exists(_POOL_ID):
    my_pool = batch_client.pool.get(_POOL_ID)
    print("Current state: {}".format(my_pool.allocation_state))

Current state: AllocationState.steady


In [ ]:
# def wrap_commands_in_shell(ostype, commands):
#     """Wrap commands in a shell
#     :param list commands: list of commands to wrap
#     :param str ostype: OS type, linux or windows
#     :rtype: str
#     :return: a shell wrapping commands
#     """
#     if ostype.lower() == "linux":
#         return "/bin/bash -c \"set -e; set -o pipefail; {}; wait\"".format(";".join(commands))
#     elif ostype.lower() == "windows":
#         return "cmd.exe /c {}".format("&".join(commands))
#     else:
#         raise ValueError("unknown ostype: {}".format(ostype))
        
# input_files = _ANALYSIS_GRIDS
# _JOB_ID = "radjob"

# print("batch_service_client:\n{0:}\n\njob_id:\n{1:}\n\ninput_files:\n{2:}\n\noutput_container_name:\n{3:}\n\noutput_container_sas_token:\n{4:}\n".format(batch_client, _JOB_ID, input_files, _JOB_NAME, output_container_sas_token))

# print("Each task will have the following passed to it ... \n")

# for input_file in input_files:
#     print("New task ...")
    
#     command = [
#         'sudo bash',
#         'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/batch tgerrish/bhrad python RunHoneybeeRadiance.py -sm ./batch/sky_mtx.json -s ./batch/surfaces.json -p ./batch/{0:}'.format(input_file.file_path),
#         'python copy_to_blob.py --filepath ./batch/{0:} --blobname {0:} --storageaccount {1:} --storagecontainer {2:} --sastoken "{3:}"'.format(input_file.file_path.replace(".json", "_result.json"), _STORAGE_ACCOUNT_NAME, _JOB_NAME, output_container_sas_token)
#     ]
    
# #     print(input_file.file_path)
    
# #     print()
    
#     print("\n####################\n".join(command))
    
#     print()

# wrap_commands_in_shell('linux', command)

In [15]:
# Add jobs to the pool

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def create_job(batch_service_client, job_id, pool_id):
    """
    Creates a job with the specified ID, associated with the specified pool.
    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str job_id: The ID for the job.
    :param str pool_id: The ID for the pool.
    """
    print('Creating job [{0:}]...'.format(job_id))

    job = batch.models.JobAddParameter(job_id, batch.models.PoolInformation(pool_id=pool_id))

    try:
        batch_service_client.job.add(job)
    except batchmodels.batch_error.BatchErrorException as err:
        print_batch_exception(err)
        raise

def add_tasks(batch_service_client, job_id, input_files, output_container_name, output_container_sas_token):
    """
    Adds a task for each input file in the collection to the specified job.

    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str job_id: The ID of the job to which to add the tasks.
    :param list input_files: A collection of input files. One task will be created for each input file.
    :param output_container_name: The ID of an Azure Blob storage container to which the tasks will upload their results.
    :param output_container_sas_token: A SAS token granting write access to the specified Azure Blob storage container.
    """

    print('Adding {} tasks to job [{}]...'.format(len(input_files), job_id))

    user = batchmodels.UserIdentity(auto_user=batchmodels.AutoUserSpecification(elevation_level=batchmodels.ElevationLevel.admin, scope=batchmodels.AutoUserScope.task))

    tasks = []

    for idx, input_file in enumerate(input_files):

        command = [
            'sudo bash',
            'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/batch tgerrish/bhrad python RunHoneybeeRadiance.py -sm ./batch/sky_mtx.json -s ./batch/surfaces.json -p ./batch/{0:}'.format(input_file.file_path),
            'python copy_to_blob.py --filepath ./batch/{0:} --blobname {0:} --storageaccount {1:} --storagecontainer {2:} --sastoken "{3:}"'.format(input_file.file_path.replace(".json", "_result.json"), _STORAGE_ACCOUNT_NAME, _JOB_NAME, output_container_sas_token)
        ]

        tasks.append(
            batch.models.TaskAddParameter(
                'Task_{0:}'.format(idx),
                wrap_commands_in_shell('linux', command),
                resource_files=[input_file],
                user_identity=user
            )
        )
    
    batch_service_client.task.add_collection(job_id, tasks)

def wait_for_tasks_to_complete(batch_service_client, job_id, timeout):
    """
    Returns when all tasks in the specified job reach the Completed state.

    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str job_id: The id of the job whose tasks should be to monitored.
    :param timedelta timeout: The duration to wait for task completion. If all
    tasks in the specified job do not reach Completed state within this time
    period, an exception will be raised.
    """
    timeout_expiration = datetime.datetime.now() + timeout

    print("Monitoring all tasks for 'Completed' state, timeout in {}...".format(timeout), end='')

    while datetime.datetime.now() < timeout_expiration:
        print('.', end='')
        sys.stdout.flush()
        tasks = [batch_service_client.task.list(job) for job in job_id]

        tasks = [item for sublist in tasks for item in sublist]

        incomplete_tasks = [task for task in tasks if task.state != batchmodels.TaskState.completed]
        if not incomplete_tasks:
            print()
            return True
        else:
            time.sleep(1)

    print()
    raise RuntimeError("ERROR: Tasks did not reach 'Completed' state within timeout period of " + str(timeout))

jobs = []
_JOB_ID = "radjob"

for i, (files, names) in enumerate(zip(list(chunks(_ANALYSIS_GRIDS, 100)),list(chunks(_ANALYSIS_GRIDS_NAMES, 100)))):
    job_id = _JOB_ID + "_" + str(i)
    jobs.append(job_id)

    # Create the job that will run the tasks.
    create_job(batch_client, job_id, _POOL_ID)

    # Add the tasks to the job. We need to supply a container shared access
    # signature (SAS) token for the tasks so that they can upload their output
    # to Azure Storage.
    add_tasks(batch_client, job_id, files, _JOB_NAME, _STORAGE_ACCOUNT_KEY) #  output_container_sas_token)


# Pause execution until tasks reach Completed state.
#wait_for_tasks_to_complete(batch_client, jobs, datetime.timedelta(hours=24))

#print("Success! All tasks reached the 'Completed' state within the specified timeout period.")

Creating job [radjob_0]...
Adding 4 tasks to job [radjob_0]...
Success! All tasks reached the 'Completed' state within the specified timeout period.


In [97]:
# Generate tasks
tasks = []
for idx, input_file in enumerate(_ANALYSIS_GRIDS):
    container_url = "https://{0:}.blob.core.windows.net/{1:}/?{3:}".format(_STORAGE_ACCOUNT_NAME, _JOB_NAME, input_file.file_path.replace(".json", ""), output_container_sas_token)
    command = [
        'sudo bash',
        'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/batch tgerrish/bhrad python RunHoneybeeRadiance.py -sm ./batch/sky_mtx.json -s ./batch/surfaces.json -p ./batch/{0:}'.format(input_file.file_path),
        #'python copy_to_blob.py --filepath ./batch/{0:} --blobname {0:} --storageaccount {1:} --storagecontainer {2:} --sastoken "{3:}"'.format(input_file.file_path.replace(".json", "_result.json"), _STORAGE_ACCOUNT_NAME, _JOB_NAME, output_container_sas_token)
    ]

    temp_task = batch.models.TaskAddParameter(
        'Task_{0:}'.format(idx), 
        wrap_commands_in_shell(
            'linux', 
            command
        ), 
        display_name=input_file.file_path, 
        container_settings=None, 
        exit_conditions=None, 
        resource_files=[input_file], 
        output_files=batch.models.OutputFile(
            input_file.file_path.replace(".json", "_result.json"), 
            batch.models.OutputFileDestination(
                container=batch.models.OutputFileBlobContainerDestination(
                    container_url, 
                    path=None)
            ), 
            batch.models.OutputFileUploadOptions("taskCompletion")
        ), 
        environment_settings=None, 
        affinity_info=None, 
        constraints=None, 
        user_identity=batch.models.UserIdentity(
            auto_user=batch.models.AutoUserSpecification(
                elevation_level=batch.models.ElevationLevel.admin, 
                scope=batch.models.AutoUserScope.task
            )
        ), 
        multi_instance_settings=None, 
        depends_on=None, 
        application_package_references=None, 
        authentication_token_settings=None
    )

    tasks.append(temp_task)

# Create job
# create_job(batch_client, "tempjobid", _POOL_ID)

# Add tasks to job
#batch_client.task.add_collection("tempjobid", tasks)



batch_client.task.add_collection("tempjobid", batch.models.TaskAddCollectionParameter(tasks))

TypeError: 'TaskAddCollectionParameter' object is not iterable

In [91]:
batch.models.OutputFileBlobContainerDestination(container_url, path=None).__dict__

{'additional_properties': {},
 'path': None,
 'container_url': 'https://radfiles.blob.core.windows.net/0000000-testjob-3513/zone4?se=2018-08-22T13%3A04%3A00Z&sp=w&sv=2017-04-17&sr=c&sig=h9K8klrifqXQUpM7q54LYTNuf7H8/qPPXSEDRyJe41Y%3D'}

In [42]:
print("add_tasks({0:}, {1:}, {2:}, {3:}, {4:})".format(batch_client, job_id, _ANALYSIS_GRIDS, _JOB_NAME, output_container_sas_token))

print()

print("user: {0:}\n\n".format(batchmodels.UserIdentity(auto_user=batchmodels.AutoUserSpecification(elevation_level=batchmodels.ElevationLevel.admin, scope=batchmodels.AutoUserScope.task))))

for idx, input_file in enumerate(_ANALYSIS_GRIDS):
    print("batch.models.TaskAddParameter(\n\t{0:},\n\twrap_commands_in_shell('linux', COMMAND),\n\tdisplay_name={2:},\n\tresource_files=[{1:}],\n\tuser_identity=USER,\n\toutput_files={3:})\n\n".format('Task_{0:}'.format(idx), input_file, input_file.file_path.replace(".json", ""), batchmodels.OutputFile()input_file.file_path.replace(".json", "_result.json")))

add_tasks(<azure.batch.batch_service_client.BatchServiceClient object at 0x000002A3C3E56FD0>, radjob_0, [<azure.batch.models.resource_file.ResourceFile object at 0x000002A3C3EC5F98>, <azure.batch.models.resource_file.ResourceFile object at 0x000002A3C3EC5E80>, <azure.batch.models.resource_file.ResourceFile object at 0x000002A3C3EC5E48>, <azure.batch.models.resource_file.ResourceFile object at 0x000002A3C3EC5EB8>], 0000000-testjob-3513, se=2018-08-22T13%3A04%3A00Z&sp=w&sv=2017-04-17&sr=c&sig=h9K8klrifqXQUpM7q54LYTNuf7H8/qPPXSEDRyJe41Y%3D)

user: {'additional_properties': {}, 'user_name': None, 'auto_user': <azure.batch.models.auto_user_specification.AutoUserSpecification object at 0x000002A3C4025160>}


batch.models.TaskAddParameter(
	Task_0,
	wrap_commands_in_shell('linux', COMMAND),
	display_name=zone1,
	resource_files=[{'additional_properties': {}, 'blob_source': 'https://radfiles.blob.core.windows.net/0000000-testjob-3513/zone1.json?se=2018-08-22T13%3A03%3A53Z&sp=r&sv=2017-04-17&sr=

In [ ]:
def add_tasks(batch_service_client, job_id, input_files, output_container_name, output_container_sas_token):
    """
    Adds a task for each input file in the collection to the specified job.

    :param batch_service_client: A Batch service client.
    :type batch_service_client: `azure.batch.BatchServiceClient`
    :param str job_id: The ID of the job to which to add the tasks.
    :param list input_files: A collection of input files. One task will be created for each input file.
    :param output_container_name: The ID of an Azure Blob storage container to which the tasks will upload their results.
    :param output_container_sas_token: A SAS token granting write access to the specified Azure Blob storage container.
    """

    print('Adding {} tasks to job [{}]...'.format(len(input_files), job_id))

    user = batchmodels.UserIdentity(auto_user=batchmodels.AutoUserSpecification(elevation_level=batchmodels.ElevationLevel.admin, scope=batchmodels.AutoUserScope.task))

    tasks = []

    for idx, input_file in enumerate(input_files):

        command = [
            'sudo bash',
            'sudo docker run --rm -v $AZ_BATCH_TASK_WORKING_DIR:/batch tgerrish/bhrad python RunHoneybeeRadiance.py -sm ./batch/sky_mtx.json -s ./batch/surfaces.json -p ./batch/{0:}'.format(input_file.file_path),
            'python copy_to_blob.py --filepath ./batch/{0:} --blobname {0:} --storageaccount {1:} --storagecontainer {2:} --sastoken "{3:}"'.format(input_file.file_path.replace(".json", "_result.json"), _STORAGE_ACCOUNT_NAME, _JOB_NAME, output_container_sas_token)
        ]

        tasks.append(
            batch.models.TaskAddParameter(
                'Task_{0:}'.format(idx),
                wrap_commands_in_shell('linux', command),
                resource_files=[input_file],
                user_identity=user
            )
        )
    
    batch_service_client.task.add_collection(job_id, tasks)

In [23]:
tasks = [batch_client.task.list(job) for job in job_id]
# tasks = [item for sublist in tasks for item in sublist]
tasks[5].__dict__

{'next_link': '',
 'current_page': [],
 '_current_page_iter_index': 0,
 '_derserializer': <msrest.serialization.Deserializer at 0x2a3c4011208>,
 '_get_next': <function azure.batch.operations.task_operations.TaskOperations.list.<locals>.internal_paging(next_link=None, raw=False)>,
 '_response': None,
 '_raw_headers': None}

In [14]:
for i, (files, names) in enumerate(zip(list(chunks(_ANALYSIS_GRIDS, 100)),list(chunks(_ANALYSIS_GRIDS_NAMES, 100)))):
    print(names)

['zone1', 'zone2', 'zone3', 'zone4']


In [ ]:
# KILL ALL PROCESSES! - TIDY UP AND NEVER LOOK BACK!

# Delete container
blob_client.delete_container(_JOB_NAME)
print("Container [{0:}] deleted".format(_JOB_NAME))

# Delete pool
batch_client.pool.delete(_POOL_ID)
print("Pool [{0:}] deleted".format(_POOL_ID))

# Delete job/s
for _JOB_ID in jobs:
    batch_client.job.delete(_JOB_ID)
    print("Job [{0:}] deleted".format(_JOB_ID))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import json

_results_files = [
    r"C:\Users\tgerrish\Documents\GitHub\SAMAzure\TestFiles\Azure\docker\azbatchworkingdir\zone4_result.json",
    r"C:\Users\tgerrish\Documents\GitHub\SAMAzure\TestFiles\Azure\docker\azbatchworkingdir\zone1_result.json",
    r"C:\Users\tgerrish\Documents\GitHub\SAMAzure\TestFiles\Azure\docker\azbatchworkingdir\zone2_result.json",
    r"C:\Users\tgerrish\Documents\GitHub\SAMAzure\TestFiles\Azure\docker\azbatchworkingdir\zone3_result.json"
]

_results = []
for _file in _results_files:
    with open(_file) as f:
        _results.append(json.load(f))

# Combine into one set of data
xs = [i["x"] for i in _results]
ys = [i["y"] for i in _results]
vals = [i["df"] for i in _results]
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.scatter(xs, ys, c=vals, cmap="viridis")
